In [25]:
MARKET = 'SNP'

In [26]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [27]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [28]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [29]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [36]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['dfrq.pkl',
 'df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

In [37]:
df_und_prices

,symbol,secType,localSymbol,conId,strike,expiry,right,contract,time,model,bid,ask,close,last,price,iv
0,ADM,STK,ADM,4165,0.0,,,"Contract(secType='STK', conId=4165, symbol='AD...",2020-11-14 03:06:19.263667+00:00,None,-1.0,-1.0,49.33,49.6,49.6,0.243515


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
375   NaN
376   NaN
377   NaN
378   NaN
379   NaN
Name: symbol, Length: 380, dtype: float64

# Refactoring Edge

In [7]:
from engine import qualify, executeAsync, margin, save_df, price
from support import Timer, quick_pf
from dfrq import get_dfrq

In [8]:
%%time
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_dfrq.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# . start the time
edge_time = Timer('dfrqs')
edge_time.start()

# * PREPARE FILES AND GET dfrq
df_opts = pd.read_pickle(DATAPATH.joinpath('df_opts.pkl'))
dfrq = get_dfrq(MARKET)

Wall time: 697 ms


In [9]:
dfrq

,symbol,undPrice,lot,gross,grosspos,remq,status
108,ITC,185.30,3200,592960.0,2.099200e+09,0,harvest
0,SHREECEM,22867.55,50,1143377.5,NaN,1,fresh
1,WIPRO,346.75,3200,1109600.0,NaN,1,fresh
2,APOLLOHOS,2195.35,500,1097675.0,NaN,1,fresh
3,JINDALSTE,219.15,5000,1095750.0,NaN,1,fresh
...,...,...,...,...,...,...,...
134,CANBK,94.35,5400,509490.0,NaN,2,fresh
135,RELIANCE,1997.20,250,499300.0,NaN,2,fresh
136,INFRATEL,176.90,2800,495320.0,NaN,2,fresh
137,IBULHSGFI,159.45,3100,494295.0,NaN,2,fresh


In [10]:
# .edge symbols are `fresh` ones from dfrq
fresh = set(dfrq[dfrq.status == 'fresh'].symbol)

# * BUILD THE EDGE

# get underlying fresh contracts
und_cts = df_unds[df_unds.symbol.isin(fresh)].contract

In [11]:
%%time
# get price and volatility for these fresh contracts
with IB().connect(HOST, PORT, CID) as ib:
    df_piv = ib.run(executeAsync(ib, price, und_cts, save_df, CONCURRENT=40, TIMEOUT=8))


Done price for ['TATACHEM0.0'] 1 out of 138. Pending ['BIOCON0.0', 'MM0.0']

Done price for ['BAJFINANC0.0', 'TCS0.0'] 40 out of 138. Pending ['POWERGRID0.0', 'NESTLEIND0.0']

Done price for ['PVR0.0', 'ICICIPRUL0.0'] 50 out of 138. Pending ['NESTLEIND0.0', 'GMRINFRA0.0']

Done price for ['MANAPPURA0.0', 'CONCOR0.0'] 80 out of 138. Pending ['AMARAJABA0.0', 'BHARTIART0.0']

Done price for ['TATAPOWER0.0', 'SBIN0.0'] 103 out of 138. Pending ['JUBLFOOD0.0', 'SHREECEM0.0']

Done price for ['LT0.0', 'NIFTY500.0'] 120 out of 138. Pending ['SBILIFE0.0', 'COFORGE0.0']

Done price for ['NTPC0.0', 'GODREJCP0.0'] 126 out of 138. Pending ['SBILIFE0.0', 'COFORGE0.0']
Wall time: 34 s


In [21]:
df_unds[['symbol', 'undPrice', 'impliedVolatility']].sort_values('symbol')

,symbol,undPrice,impliedVolatility
86,ACC,1670.15,0.330521
67,ADANIENT,360.10,0.538020
77,ADANIPORT,371.60,0.399843
81,AMARAJABA,826.00,0.375614
133,AMBUJACEM,259.70,0.361571
...,...,...,...
134,UPL,435.70,0.471858
26,VEDL,101.90,0.515498
10,VOLTAS,760.70,0.341345
70,WIPRO,346.75,0.347996


In [24]:
df_piv[['symbol', 'price', 'iv', 'last', 'close']].sort_values('symbol')

,symbol,price,iv,last,close
5,ACC,1674.90,0.328341,NaN,1674.90
19,ADANIENT,365.55,0.532844,NaN,365.55
11,ADANIPORT,368.05,0.396530,NaN,368.05
130,AMARAJABA,834.95,0.377056,NaN,834.95
126,AMBUJACEM,262.85,0.362073,NaN,262.85
...,...,...,...,...,...
87,UPL,429.60,0.451669,NaN,429.60
96,VEDL,103.45,0.511616,NaN,103.45
61,VOLTAS,755.15,0.347707,NaN,755.15
7,WIPRO,345.55,0.352989,NaN,345.55
